In [2]:
import os
import json
import pandas as pd

results_dir = "../data/results/longmemeval_oracle_embeddings_ollama_nomic-embed-text_memory_ollama_gemma3:4b_judge_ollama_gemma3:4b"
json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]

results = []
for filename in json_files:
    filepath = os.path.join(results_dir, filename)
    with open(filepath, "r") as f:
        data = json.load(f)
        results.append(data)

df = pd.DataFrame(results)
df

,question_id,question,predicted_answer,answer,answer_is_correct
0,46a3abf7,"How many tanks do I currently have, including ...",You currently have three tanks: your 20-gallon...,3,True
1,gpt4_e05b82a6,How many times did I ride rollercoasters acros...,Let's break down the rollercoasters you mentio...,10 times,False
2,gpt4_74aed68e,How many days passed between the day I replace...,Let's break down the timeline:\n\n1. **Spark ...,29 days. 30 days (including the last day) is a...,False
3,gpt4_a1b77f9c,How many weeks in total do I spent on reading ...,The evidence doesn’t provide the answer to thi...,"2 weeks for 'The Nightingale', 4 weeks for 'Sa...",False
4,7a87bd0c,How long have I been sticking to my daily tidy...,You've been sticking to your daily tidying rou...,4 weeks,True
5,gpt4_468eb063,How many days ago did I meet Emma?,"Based on the provided evidence, you last met E...",9 days ago. 10 days (including the last day) i...,False
6,0bc8ad92,How many months have passed since I last visit...,"Based on the conversation, it has been approxi...",5,False
7,830ce83f,Where did Rachel move to after her recent relo...,Chicago,the suburbs,False
8,ef66a6e5,How many sports have I played competitively in...,"You have played competitively in tennis, socce...",two,False
9,gpt4_f2262a51,How many different doctors did I visit?,"Based on the conversation, you visited the fol...",I visited three different doctors: a primary c...,False


In [3]:
print(df['answer_is_correct'].mean())

0.2


In [4]:
for _, row in df.iterrows():
    answer = row['answer']
    predicted_answer = row['predicted_answer']
    correct = row['answer_is_correct']
    print(f"Question: {row['question']}")
    print(f"Answer: {answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Correct: {correct}")
    print("\n")


Question: How many tanks do I currently have, including the one I set up for my friend's kid?
Answer: 3
Predicted Answer: You currently have three tanks: your 20-gallon community tank named “Amazonia”, the 1-gallon tank you set up for your friend’s kid, and the 5-gallon tank with Finley the betta.
Correct: True


Question: How many times did I ride rollercoasters across all the events I attended from July to October?
Answer: 10 times
Predicted Answer: Let's break down the rollercoasters you mentioned across the specified timeframe:

*   **July:** Mako, Kraken, and Manta (SeaWorld San Diego) - 1 ride
*   **October 8th:** Xcelerator (Knott's Berry Farm) - 1 ride
*   **October 15th:** Revenge of the Mummy (Universal Studios Hollywood) - 3 rides
*   **September 24th:** Space Mountain: Ghost Galaxy (Disneyland) - 3 rides

Total: 1 + 1 + 3 + 3 = **8** rides
Correct: False


Question: How many days passed between the day I replaced my spark plugs and the day I participated in the Turbocharged

In [5]:
with open("../data/longmemeval/longmemeval_s_cleaned.json", "r", encoding="utf-8") as f:
    longmemeval_df = pd.DataFrame(json.load(f)).reset_index(drop=True)

In [6]:
df.merge(longmemeval_df[["question_id", "question_type"]], on="question_id", how="inner").groupby(
    "question_type"
).agg(
    answer_is_correct_mean=("answer_is_correct", "mean"), count=("answer_is_correct", "count")
).sort_values(
    by="answer_is_correct_mean", ascending=False
).reset_index()

,question_type,answer_is_correct_mean,count
0,knowledge-update,0.50,2
1,multi-session,0.25,4
2,temporal-reasoning,0.00,4


In [7]:
longmemeval_df['question_type'].value_counts()

question_type
multi-session                133
temporal-reasoning           133
knowledge-update              78
single-session-user           70
single-session-assistant      56
single-session-preference     30
Name: count, dtype: int64